# Comprehensive SQL Interview Prep for AI/ML & Data Science

## Table of Contents
1. [SQL Fundamentals](#sql-fundamentals)
2. [Data Retrieval & Filtering](#data-retrieval--filtering)
3. [Joins & Relationships](#joins--relationships)
4. [Aggregations & Grouping](#aggregations--grouping)
5. [Window Functions](#window-functions)
6. [Subqueries & CTEs](#subqueries--ctes)
7. [Data Manipulation](#data-manipulation)
8. [Performance & Optimization](#performance--optimization)
9. [Advanced Topics for ML/DS](#advanced-topics-for-mlds)
10. [Common Interview Questions](#common-interview-questions)

---

## SQL Fundamentals

### Database Concepts
- **Database**: Organized collection of structured data
- **Table**: Collection of related data with rows (records) and columns (fields)
- **Primary Key**: Unique identifier for each row
- **Foreign Key**: Links rows between tables
- **Schema**: Structure/blueprint of database

### Basic Data Types
- **Numeric**: INT, BIGINT, DECIMAL, FLOAT, DOUBLE
- **String**: CHAR, VARCHAR, TEXT
- **Date/Time**: DATE, TIME, DATETIME, TIMESTAMP
- **Boolean**: BOOLEAN (TRUE/FALSE)
- **NULL**: Represents missing/unknown values

### Basic Query Structure
```sql
SELECT column1, column2
FROM table_name
WHERE condition
ORDER BY column1
LIMIT 10;
```

---

## Data Retrieval & Filtering

### SELECT Statement Basics
```sql
-- Select all columns
SELECT * FROM users;

-- Select specific columns
SELECT user_id, name, email FROM users;

-- Distinct values (remove duplicates)
SELECT DISTINCT country FROM users;

-- Column aliases
SELECT user_id AS id, name AS user_name FROM users;
```

### WHERE Clause Operators
```sql
-- Comparison operators: =, !=, <, >, <=, >=
SELECT * FROM products WHERE price > 100;

-- Logical operators: AND, OR, NOT
SELECT * FROM products WHERE price > 50 AND category = 'Electronics';

-- IN operator
SELECT * FROM users WHERE country IN ('USA', 'Canada', 'UK');

-- BETWEEN operator
SELECT * FROM orders WHERE order_date BETWEEN '2024-01-01' AND '2024-12-31';

-- LIKE operator (pattern matching)
SELECT * FROM users WHERE email LIKE '%@gmail.com';
-- % matches any sequence of characters
-- _ matches single character

-- IS NULL / IS NOT NULL
SELECT * FROM users WHERE phone_number IS NULL;
```

### Sorting Results
```sql
-- ORDER BY (default is ASC)
SELECT * FROM products ORDER BY price DESC;

-- Multiple columns
SELECT * FROM products ORDER BY category ASC, price DESC;
```

### Limiting Results
```sql
-- LIMIT (MySQL, PostgreSQL)
SELECT * FROM users LIMIT 10;

-- LIMIT with OFFSET
SELECT * FROM users LIMIT 10 OFFSET 20;  -- Skip first 20, get next 10

-- TOP (SQL Server)
SELECT TOP 10 * FROM users;
```

---

## Joins & Relationships

### Types of Joins

#### INNER JOIN
Returns only matching rows from both tables
```sql
SELECT u.user_id, u.name, o.order_id, o.total
FROM users u
INNER JOIN orders o ON u.user_id = o.user_id;
```

#### LEFT JOIN (LEFT OUTER JOIN)
Returns all rows from left table, matching rows from right (NULL if no match)
```sql
SELECT u.user_id, u.name, o.order_id
FROM users u
LEFT JOIN orders o ON u.user_id = o.user_id;
```

#### RIGHT JOIN (RIGHT OUTER JOIN)
Returns all rows from right table, matching rows from left
```sql
SELECT u.user_id, u.name, o.order_id
FROM users u
RIGHT JOIN orders o ON u.user_id = o.user_id;
```

#### FULL OUTER JOIN
Returns all rows from both tables (NULL where no match)
```sql
SELECT u.user_id, u.name, o.order_id
FROM users u
FULL OUTER JOIN orders o ON u.user_id = o.user_id;
```

#### CROSS JOIN
Cartesian product (all possible combinations)
```sql
SELECT * FROM colors CROSS JOIN sizes;
```

#### SELF JOIN
Join table to itself
```sql
-- Find employees and their managers
SELECT e.name AS employee, m.name AS manager
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.employee_id;
```

### Multiple Joins
```sql
SELECT u.name, o.order_id, p.product_name, oi.quantity
FROM users u
INNER JOIN orders o ON u.user_id = o.user_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
INNER JOIN products p ON oi.product_id = p.product_id;
```

---

## Aggregations & Grouping

### Aggregate Functions
```sql
-- COUNT: Count rows
SELECT COUNT(*) FROM users;
SELECT COUNT(DISTINCT country) FROM users;

-- SUM: Total of numeric column
SELECT SUM(total) FROM orders;

-- AVG: Average value
SELECT AVG(price) FROM products;

-- MIN/MAX: Minimum/Maximum value
SELECT MIN(price), MAX(price) FROM products;
```

### GROUP BY
Groups rows with same values for aggregation
```sql
-- Count users per country
SELECT country, COUNT(*) AS user_count
FROM users
GROUP BY country;

-- Total sales by category
SELECT category, SUM(price * quantity) AS total_sales
FROM products
GROUP BY category;

-- Multiple grouping columns
SELECT country, city, COUNT(*) AS user_count
FROM users
GROUP BY country, city;
```

### HAVING Clause
Filters groups (WHERE filters rows before grouping)
```sql
-- Countries with more than 100 users
SELECT country, COUNT(*) AS user_count
FROM users
GROUP BY country
HAVING COUNT(*) > 100;

-- Categories with average price > 50
SELECT category, AVG(price) AS avg_price
FROM products
GROUP BY category
HAVING AVG(price) > 50;
```

### GROUP BY vs WHERE vs HAVING
```sql
SELECT category, COUNT(*) AS product_count, AVG(price) AS avg_price
FROM products
WHERE price > 10              -- Filter rows BEFORE grouping
GROUP BY category
HAVING COUNT(*) > 5;          -- Filter groups AFTER grouping
```

---

## Window Functions

Window functions perform calculations across rows related to current row without collapsing results.

### ROW_NUMBER()
Assigns unique sequential number to rows
```sql
SELECT 
    user_id, 
    name, 
    score,
    ROW_NUMBER() OVER (ORDER BY score DESC) AS rank
FROM users;
```

### RANK() and DENSE_RANK()
```sql
-- RANK: Same values get same rank, gaps in sequence
-- DENSE_RANK: Same values get same rank, no gaps
SELECT 
    user_id,
    score,
    RANK() OVER (ORDER BY score DESC) AS rank,
    DENSE_RANK() OVER (ORDER BY score DESC) AS dense_rank
FROM users;
-- Score: 100, 95, 95, 90
-- RANK: 1, 2, 2, 4
-- DENSE_RANK: 1, 2, 2, 3
```

### PARTITION BY
Divides result set into partitions
```sql
-- Rank users within each country
SELECT 
    user_id, 
    name, 
    country,
    score,
    RANK() OVER (PARTITION BY country ORDER BY score DESC) AS country_rank
FROM users;
```

### LAG() and LEAD()
Access data from previous/next rows
```sql
-- Compare with previous row
SELECT 
    order_date,
    total,
    LAG(total, 1) OVER (ORDER BY order_date) AS prev_total,
    total - LAG(total, 1) OVER (ORDER BY order_date) AS difference
FROM orders;

-- Look ahead to next row
SELECT 
    order_date,
    total,
    LEAD(total, 1) OVER (ORDER BY order_date) AS next_total
FROM orders;
```

### Running Totals and Moving Averages
```sql
-- Running total
SELECT 
    order_date,
    total,
    SUM(total) OVER (ORDER BY order_date) AS running_total
FROM orders;

-- Moving average (last 3 rows)
SELECT 
    order_date,
    total,
    AVG(total) OVER (ORDER BY order_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_avg_3
FROM orders;
```

### NTILE()
Divides rows into specified number of groups
```sql
-- Divide users into 4 quartiles by score
SELECT 
    user_id,
    score,
    NTILE(4) OVER (ORDER BY score DESC) AS quartile
FROM users;
```

### Common Window Function Use Cases for DS/ML
```sql
-- Calculate percentile rank
SELECT 
    user_id,
    score,
    PERCENT_RANK() OVER (ORDER BY score) AS percentile
FROM users;

-- Find top N per group
WITH ranked AS (
    SELECT 
        category,
        product_name,
        price,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC) AS rn
    FROM products
)
SELECT * FROM ranked WHERE rn <= 3;
```

---

## Subqueries & CTEs

### Subqueries (Nested Queries)

#### Scalar Subquery (returns single value)
```sql
-- Find users with above-average score
SELECT name, score
FROM users
WHERE score > (SELECT AVG(score) FROM users);
```

#### Column Subquery (returns single column)
```sql
-- Find users who made orders
SELECT name
FROM users
WHERE user_id IN (SELECT DISTINCT user_id FROM orders);
```

#### Row Subquery
```sql
-- Find products with max price per category
SELECT *
FROM products
WHERE (category, price) IN (
    SELECT category, MAX(price)
    FROM products
    GROUP BY category
);
```

#### Table Subquery (in FROM clause)
```sql
SELECT category, avg_price
FROM (
    SELECT category, AVG(price) AS avg_price
    FROM products
    GROUP BY category
) AS category_stats
WHERE avg_price > 100;
```

#### Correlated Subquery
Subquery references outer query
```sql
-- Find users with score above their country average
SELECT u1.name, u1.country, u1.score
FROM users u1
WHERE u1.score > (
    SELECT AVG(u2.score)
    FROM users u2
    WHERE u2.country = u1.country
);
```

### Common Table Expressions (CTEs)

CTEs improve readability and reusability using WITH clause

#### Basic CTE
```sql
WITH high_value_customers AS (
    SELECT user_id, SUM(total) AS total_spent
    FROM orders
    GROUP BY user_id
    HAVING SUM(total) > 1000
)
SELECT u.name, hvc.total_spent
FROM users u
INNER JOIN high_value_customers hvc ON u.user_id = hvc.user_id;
```

#### Multiple CTEs
```sql
WITH 
customer_stats AS (
    SELECT user_id, COUNT(*) AS order_count, SUM(total) AS total_spent
    FROM orders
    GROUP BY user_id
),
product_stats AS (
    SELECT product_id, COUNT(*) AS times_ordered
    FROM order_items
    GROUP BY product_id
)
SELECT * FROM customer_stats cs
INNER JOIN product_stats ps ON cs.user_id = ps.product_id;
```

#### Recursive CTE
Useful for hierarchical data
```sql
-- Find all employees under a manager (organizational hierarchy)
WITH RECURSIVE employee_hierarchy AS (
    -- Base case: start with CEO
    SELECT employee_id, name, manager_id, 1 AS level
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursive case: find direct reports
    SELECT e.employee_id, e.name, e.manager_id, eh.level + 1
    FROM employees e
    INNER JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
)
SELECT * FROM employee_hierarchy;
```

---

## Data Manipulation

### INSERT
```sql
-- Insert single row
INSERT INTO users (name, email, country)
VALUES ('John Doe', 'john@example.com', 'USA');

-- Insert multiple rows
INSERT INTO users (name, email, country)
VALUES 
    ('Alice', 'alice@example.com', 'UK'),
    ('Bob', 'bob@example.com', 'Canada');

-- Insert from SELECT
INSERT INTO archived_orders
SELECT * FROM orders WHERE order_date < '2023-01-01';
```

### UPDATE
```sql
-- Update single column
UPDATE users
SET email = 'newemail@example.com'
WHERE user_id = 123;

-- Update multiple columns
UPDATE products
SET price = price * 1.1, updated_at = CURRENT_TIMESTAMP
WHERE category = 'Electronics';

-- Update with JOIN (MySQL/PostgreSQL)
UPDATE users u
INNER JOIN orders o ON u.user_id = o.user_id
SET u.last_order_date = o.order_date
WHERE o.order_id = (
    SELECT MAX(order_id) FROM orders WHERE user_id = u.user_id
);
```

### DELETE
```sql
-- Delete specific rows
DELETE FROM users WHERE user_id = 123;

-- Delete with condition
DELETE FROM orders WHERE order_date < '2023-01-01';

-- Delete all rows (keeps table structure)
DELETE FROM temp_table;

-- TRUNCATE (faster, resets auto-increment)
TRUNCATE TABLE temp_table;
```

### UPSERT (INSERT ON DUPLICATE KEY)
```sql
-- MySQL
INSERT INTO users (user_id, name, email)
VALUES (123, 'John', 'john@example.com')
ON DUPLICATE KEY UPDATE 
    name = VALUES(name),
    email = VALUES(email);

-- PostgreSQL
INSERT INTO users (user_id, name, email)
VALUES (123, 'John', 'john@example.com')
ON CONFLICT (user_id) 
DO UPDATE SET 
    name = EXCLUDED.name,
    email = EXCLUDED.email;
```

---

## Performance & Optimization

### Indexes

Indexes speed up data retrieval but slow down writes

```sql
-- Create index on single column
CREATE INDEX idx_users_email ON users(email);

-- Create composite index
CREATE INDEX idx_orders_user_date ON orders(user_id, order_date);

-- Unique index
CREATE UNIQUE INDEX idx_users_email_unique ON users(email);

-- Drop index
DROP INDEX idx_users_email;
```

**When to use indexes:**
- Columns in WHERE clauses
- Columns in JOIN conditions
- Columns in ORDER BY
- Foreign keys

**When NOT to use indexes:**
- Small tables
- Columns with frequent updates
- Columns with low cardinality (few distinct values)

### EXPLAIN / EXPLAIN ANALYZE

Understand query execution plan
```sql
EXPLAIN SELECT * FROM users WHERE email = 'john@example.com';

-- PostgreSQL: actual execution statistics
EXPLAIN ANALYZE SELECT * FROM users WHERE email = 'john@example.com';
```

### Query Optimization Tips

#### Use specific columns instead of SELECT *
```sql
-- Bad
SELECT * FROM users;

-- Good
SELECT user_id, name, email FROM users;
```

#### Use EXISTS instead of COUNT for existence checks
```sql
-- Bad
SELECT * FROM users WHERE (SELECT COUNT(*) FROM orders WHERE user_id = users.user_id) > 0;

-- Good
SELECT * FROM users WHERE EXISTS (SELECT 1 FROM orders WHERE user_id = users.user_id);
```

#### Avoid functions on indexed columns in WHERE
```sql
-- Bad (can't use index)
SELECT * FROM users WHERE YEAR(created_at) = 2024;

-- Good (can use index)
SELECT * FROM users WHERE created_at >= '2024-01-01' AND created_at < '2025-01-01';
```

#### Use LIMIT for large result sets
```sql
SELECT * FROM users ORDER BY created_at DESC LIMIT 100;
```

#### Use covering indexes
```sql
-- If query only needs user_id and email, create covering index
CREATE INDEX idx_users_id_email ON users(user_id, email);

SELECT user_id, email FROM users WHERE user_id > 1000;
```

---

## Advanced Topics for ML/DS

### String Functions
```sql
-- Concatenation
SELECT CONCAT(first_name, ' ', last_name) AS full_name FROM users;

-- Length
SELECT name, LENGTH(name) FROM users;

-- Substring
SELECT SUBSTRING(email, 1, POSITION('@' IN email) - 1) AS username FROM users;

-- Replace
SELECT REPLACE(phone, '-', '') AS clean_phone FROM users;

-- Upper/Lower case
SELECT UPPER(name), LOWER(email) FROM users;

-- Trim whitespace
SELECT TRIM(name) FROM users;
```

### Date/Time Functions
```sql
-- Current date/time
SELECT CURRENT_DATE, CURRENT_TIME, CURRENT_TIMESTAMP;

-- Extract parts
SELECT 
    EXTRACT(YEAR FROM order_date) AS year,
    EXTRACT(MONTH FROM order_date) AS month,
    EXTRACT(DAY FROM order_date) AS day
FROM orders;

-- Date arithmetic
SELECT order_date, order_date + INTERVAL '7 days' AS due_date FROM orders;

-- Date difference
SELECT DATEDIFF(end_date, start_date) AS duration FROM projects;

-- Format date
SELECT DATE_FORMAT(order_date, '%Y-%m-%d') FROM orders;
```

### Mathematical Functions
```sql
-- Rounding
SELECT ROUND(price, 2), CEIL(price), FLOOR(price) FROM products;

-- Absolute value
SELECT ABS(balance) FROM accounts;

-- Power and square root
SELECT POWER(2, 3), SQRT(16);

-- Random
SELECT RANDOM();  -- PostgreSQL
SELECT RAND();    -- MySQL
```

### CASE Statements (Conditional Logic)
```sql
-- Simple CASE
SELECT 
    name,
    score,
    CASE 
        WHEN score >= 90 THEN 'A'
        WHEN score >= 80 THEN 'B'
        WHEN score >= 70 THEN 'C'
        ELSE 'F'
    END AS grade
FROM students;

-- Counted aggregation with CASE
SELECT 
    category,
    COUNT(CASE WHEN price > 100 THEN 1 END) AS expensive_count,
    COUNT(CASE WHEN price <= 100 THEN 1 END) AS affordable_count
FROM products
GROUP BY category;
```

### Pivot Tables (CASE with Aggregation)
```sql
SELECT 
    user_id,
    SUM(CASE WHEN product_category = 'Electronics' THEN amount ELSE 0 END) AS electronics,
    SUM(CASE WHEN product_category = 'Clothing' THEN amount ELSE 0 END) AS clothing,
    SUM(CASE WHEN product_category = 'Food' THEN amount ELSE 0 END) AS food
FROM orders
GROUP BY user_id;
```

### Cohort Analysis Example
```sql
WITH user_cohorts AS (
    SELECT 
        user_id,
        DATE_TRUNC('month', MIN(order_date)) AS cohort_month
    FROM orders
    GROUP BY user_id
),
cohort_orders AS (
    SELECT 
        uc.cohort_month,
        DATE_TRUNC('month', o.order_date) AS order_month,
        COUNT(DISTINCT o.user_id) AS users
    FROM orders o
    INNER JOIN user_cohorts uc ON o.user_id = uc.user_id
    GROUP BY uc.cohort_month, DATE_TRUNC('month', o.order_date)
)
SELECT 
    cohort_month,
    order_month,
    users,
    DATEDIFF('month', cohort_month, order_month) AS month_number
FROM cohort_orders
ORDER BY cohort_month, order_month;
```

### Percentiles
```sql
-- Calculate percentiles
SELECT 
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY price) AS median,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY price) AS q1,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY price) AS q3
FROM products;
```

### Time Series Analysis
```sql
-- Daily active users
SELECT 
    DATE(event_timestamp) AS date,
    COUNT(DISTINCT user_id) AS dau
FROM events
WHERE event_timestamp >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY DATE(event_timestamp)
ORDER BY date;

-- Rolling 7-day average
WITH daily_stats AS (
    SELECT 
        DATE(event_timestamp) AS date,
        COUNT(DISTINCT user_id) AS dau
    FROM events
    GROUP BY DATE(event_timestamp)
)
SELECT 
    date,
    dau,
    AVG(dau) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_7day_avg
FROM daily_stats
ORDER BY date;
```

---

## Common Interview Questions

### 1. Find Nth Highest Salary
```sql
-- Method 1: Using LIMIT and OFFSET
SELECT DISTINCT salary
FROM employees
ORDER BY salary DESC
LIMIT 1 OFFSET 2;  -- For 3rd highest (N-1)

-- Method 2: Using subquery
SELECT MAX(salary)
FROM employees
WHERE salary < (
    SELECT MAX(salary)
    FROM employees
    WHERE salary < (SELECT MAX(salary) FROM employees)
);

-- Method 3: Using DENSE_RANK (best for ties)
WITH ranked AS (
    SELECT salary, DENSE_RANK() OVER (ORDER BY salary DESC) AS rank
    FROM employees
)
SELECT DISTINCT salary
FROM ranked
WHERE rank = 3;
```

### 2. Find Duplicate Records
```sql
-- Find duplicates
SELECT email, COUNT(*) AS count
FROM users
GROUP BY email
HAVING COUNT(*) > 1;

-- Get all duplicate rows
SELECT u.*
FROM users u
INNER JOIN (
    SELECT email
    FROM users
    GROUP BY email
    HAVING COUNT(*) > 1
) duplicates ON u.email = duplicates.email;
```

### 3. Delete Duplicate Records (Keep One)
```sql
-- Using CTE and ROW_NUMBER
WITH ranked AS (
    SELECT 
        user_id,
        ROW_NUMBER() OVER (PARTITION BY email ORDER BY user_id) AS rn
    FROM users
)
DELETE FROM users
WHERE user_id IN (SELECT user_id FROM ranked WHERE rn > 1);
```

### 4. Self-Join: Find Pairs
```sql
-- Find pairs of employees in same department
SELECT 
    e1.name AS employee1,
    e2.name AS employee2,
    e1.department
FROM employees e1
INNER JOIN employees e2 
    ON e1.department = e2.department 
    AND e1.employee_id < e2.employee_id;  -- Avoid duplicates and self-pairs
```

### 5. Running Total / Cumulative Sum
```sql
SELECT 
    order_date,
    total,
    SUM(total) OVER (ORDER BY order_date) AS cumulative_total
FROM orders
ORDER BY order_date;
```

### 6. Find Gaps in Sequence
```sql
-- Find missing IDs in sequence
WITH RECURSIVE all_ids AS (
    SELECT MIN(id) AS id FROM users
    UNION ALL
    SELECT id + 1 FROM all_ids WHERE id < (SELECT MAX(id) FROM users)
)
SELECT ai.id AS missing_id
FROM all_ids ai
LEFT JOIN users u ON ai.id = u.id
WHERE u.id IS NULL;
```

### 7. Top N per Group
```sql
-- Top 3 products per category by sales
WITH ranked AS (
    SELECT 
        category,
        product_name,
        sales,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY sales DESC) AS rank
    FROM products
)
SELECT category, product_name, sales
FROM ranked
WHERE rank <= 3;
```

### 8. Year-over-Year Growth
```sql
WITH yearly_sales AS (
    SELECT 
        EXTRACT(YEAR FROM order_date) AS year,
        SUM(total) AS total_sales
    FROM orders
    GROUP BY EXTRACT(YEAR FROM order_date)
)
SELECT 
    year,
    total_sales,
    LAG(total_sales) OVER (ORDER BY year) AS prev_year_sales,
    ROUND(100.0 * (total_sales - LAG(total_sales) OVER (ORDER BY year)) / 
          LAG(total_sales) OVER (ORDER BY year), 2) AS yoy_growth_pct
FROM yearly_sales;
```

### 9. Find Consecutive Events
```sql
-- Find users with 3+ consecutive daily logins
WITH login_dates AS (
    SELECT 
        user_id,
        DATE(login_timestamp) AS login_date,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY DATE(login_timestamp)) AS rn
    FROM logins
),
consecutive_groups AS (
    SELECT 
        user_id,
        login_date,
        DATE(login_date) - INTERVAL '1 day' * rn AS group_date
    FROM login_dates
)
SELECT user_id, COUNT(*) AS consecutive_days
FROM consecutive_groups
GROUP BY user_id, group_date
HAVING COUNT(*) >= 3;
```

### 10. Pivot Data
```sql
-- Pivot monthly sales by category
SELECT 
    category,
    SUM(CASE WHEN EXTRACT(MONTH FROM order_date) = 1 THEN total ELSE 0 END) AS jan,
    SUM(CASE WHEN EXTRACT(MONTH FROM order_date) = 2 THEN total ELSE 0 END) AS feb,
    SUM(CASE WHEN EXTRACT(MONTH FROM order_date) = 3 THEN total ELSE 0 END) AS mar
FROM orders
GROUP BY category;
```

### 11. Second Order Aggregation
```sql
-- Average order value per customer
SELECT AVG(customer_total) AS avg_customer_lifetime_value
FROM (
    SELECT user_id, SUM(total) AS customer_total
    FROM orders
    GROUP BY user_id
) AS customer_totals;
```

### 12. Retention Rate
```sql
-- Calculate 30-day retention
WITH first_order AS (
    SELECT user_id, MIN(order_date) AS first_order_date
    FROM orders
    GROUP BY user_id
),
returned AS (
    SELECT DISTINCT fo.user_id
    FROM first_order fo
    INNER JOIN orders o 
        ON fo.user_id = o.user_id
        AND o.order_date BETWEEN fo.first_order_date + INTERVAL '30 days' 
                            AND fo.first_order_date + INTERVAL '37 days'
)
SELECT 
    COUNT(DISTINCT fo.user_id) AS total_customers,
    COUNT(DISTINCT r.user_id) AS retained_customers,
    ROUND(100.0 * COUNT(DISTINCT r.user_id) / COUNT(DISTINCT fo.user_id), 2) AS retention_rate
FROM first_order fo
LEFT JOIN returned r ON fo.user_id = r.user_id;
```

---

## Key Concepts for ML/DS Interviews

### Data Quality Checks
```sql
-- Check for NULL values
SELECT 
    COUNT(*) AS total_rows,
    COUNT(*) - COUNT(column_name) AS null_count,
    ROUND(100.0 * (COUNT(*) - COUNT(column_name)) / COUNT(*), 2) AS null_percentage
FROM table_name;

-- Check for duplicates
SELECT column1, column2, COUNT(*)
FROM table_name
GROUP BY column1, column2
HAVING COUNT(*) > 1;

-- Data distribution
SELECT 
    MIN(value) AS min_value,
    MAX(value) AS max_value,
    AVG(value) AS mean_value,
    STDDEV(value) AS std_dev,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY value) AS median
FROM table_name;
```

### Feature Engineering Examples
```sql
-- Create time-based features
SELECT 
    user_id,
    EXTRACT(HOUR FROM event_timestamp) AS hour_of_day,
    EXTRACT(DOW FROM event_timestamp) AS day_of_week,
    CASE WHEN EXTRACT(DOW FROM event_timestamp) IN (0, 6) THEN 1 ELSE 0 END AS is_weekend
FROM events;

-- Create aggregated features
SELECT 
    user_id,
    COUNT(*) AS total_orders,
    SUM(total) AS total_spent,
    AVG(total) AS avg_order_value,
    MAX(total) AS max_order_value,
    DATEDIFF(day, MIN(order_date), MAX(order_date)) AS customer_lifetime_days
FROM orders
GROUP BY user_id;
```

### Interview Tips
1. **Always clarify requirements** before writing queries
2. **Start with simple solutions**, then optimize
3. **Explain your thought process** as you write
4. **Test edge cases**: NULL values, empty results, duplicates
5. **Consider performance**: indexes, query complexity
6. **Know your SQL dialect** (MySQL, PostgreSQL, SQL Server differences)
7. **Practice on real datasets**: LeetCode, HackerRank, StrataScratch

---

## Quick Reference Cheat Sheet

### Order of Execution
1. FROM (including JOINs)
2. WHERE
3. GROUP BY
4. HAVING
5. SELECT
6. DISTINCT
7. ORDER BY
8. LIMIT/OFFSET

### Common Gotchas
- Can't use column aliases in WHERE (use in HAVING instead)
- NULL ≠ NULL (use IS NULL, not = NULL)
- COUNT(*) counts rows; COUNT(column) counts non-NULL values
- DISTINCT applies to entire SELECT, not individual columns
- Window functions can't be in WHERE clause

### Performance Tips
- Index columns used in WHERE, JOIN, ORDER BY
- Use specific columns, not SELECT *
- Use EXISTS instead of IN for large subqueries
- Avoid functions on indexed columns in WHERE
- Use LIMIT when testing queries
- Consider partitioning for very large tables

### Best Practices
- Use meaningful aliases
- Format queries for readability
- Comment complex logic
- Use CTEs for complex queries
- Always handle NULL values explicitly
- Test with edge cases